# Imports

In [ ]:
from collections import defaultdict

import warnings
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

from matplotlib.colors import LinearSegmentedColormap

warnings.filterwarnings("ignore")
%matplotlib inline

# Constants

In [ ]:
DATASET_PATH = '/kaggle/input/movies-dataset-for-feature-extracion-prediction/movies.csv'
REGEX_MOVIE_YEAR = r'\(\d{4}\)|\([MDCLXVI]+(?:M{0,3}D?C{0,3}L?X{0,3}V?I{0,3})\)\(\d{4}\)|\([MDCLXVI]+(?:M{0,3}D?C{0,3}L?X{0,3}V?I{0,3})\)'

GENRE_WEIGHT_WRT_RATING = 0.4
DIRECTOR_WEIGHT_WRT_RATING = 0.3
CAST_WEIGHT_WRT_RATING = 0.3

GENRE_WEIGHT_WRT_VOTES = 0.6
DIRECTOR_WEIGHT_WRT_VOTES = 0.1
CAST_WEIGHT_WRT_VOTES = 0.3

# Helpers

In [ ]:
def get_data_frame_quick_insights(dataframe):
    """
    Check basic insights, which includes:
    1. Column Type
    2. Null Count
    3. Null Percentage
    """
    # Calculate null percentages and counts
    null_percentage = (dataframe.isnull().mean() * 100).round(1)
    null_count =  dataframe.isna().sum()

    # Create a new DataFrame to display the results
    result_df = pd.DataFrame({
        'Column Type': dataframe.dtypes,
        'Null Count': null_count, 
        'Null Percentage': null_percentage.values,
    })

    # Reset the index for a cleaner DataFrame
    result_df.reset_index(inplace=True)
    result_df.rename(columns={'index': 'Column Name'}, inplace=True)

    # Return result DataFrame
    return result_df


def extract_director_and_stars_info(row):
    parts = row.split('|')
    director = 'Unknown'
    stars = 'Unknown'

    for part in parts:
        if part.startswith('Director:'):
            director = part.replace('Director:', '').strip()
        if part.startswith('Stars:'):
            stars = part.replace('Stars:', '').strip()
    
    return director, stars


def get_directors_count(movie_directors):
    director_counts = {}

    for director in movie_directors:
        if director in director_counts:
            director_counts[director] += 1
        else:
            director_counts[director] = 1
            
    df = pd.DataFrame(
        list(director_counts.items()), 
        columns=['DIRECTOR', 'COUNT']
    )
    df = df.sort_values(by='COUNT', ascending=False)
    df = df.reset_index(drop=True)
    return df


def get_casts_count(movies_cast):
    occurrence_dict = defaultdict(int)

    for movie_cast in movies_cast:
        cast_list = movie_cast.split(',')
        for name in cast_list:
            occurrence_dict[name] += 1

    df = pd.DataFrame(
        list(occurrence_dict.items()), 
        columns=['CAST', 'COUNT']
    )
    df = df.sort_values(by='COUNT', ascending=False)
    df = df.reset_index(drop=True)
    return df


def get_genre_count(dataframe):
    genre_count = {}

    genres = dataframe.str.split(', ').explode()
    genre_count = genres.value_counts().to_dict()
    
    df = pd.DataFrame(
        list(genre_count.items()), 
        columns=['GENRE', 'COUNT']
    )
    df = df.sort_values(by='COUNT', ascending=False)
    df = df.reset_index(drop=True)
    return df

def cal_movie_rating(genre_rating, director_rating, cast_rating):
    movie_rating = (
        GENRE_WEIGHT_WRT_RATING * genre_rating +
        DIRECTOR_WEIGHT_WRT_RATING * director_rating +
        CAST_WEIGHT_WRT_RATING * cast_rating
    ) / (GENRE_WEIGHT_WRT_RATING + DIRECTOR_WEIGHT_WRT_RATING + CAST_WEIGHT_WRT_RATING)

    # Ensure the movie rating is within the range of 1-10
    movie_rating = max(1, min(10, movie_rating))
    
    return max(1, min(10, movie_rating))

def cal_movie_votes(genre_votes, director_votes, cast_votes):
    movie_vote = (
        GENRE_WEIGHT_WRT_VOTES * genre_votes +
        DIRECTOR_WEIGHT_WRT_VOTES * director_votes +
        CAST_WEIGHT_WRT_VOTES * cast_votes
    ) / (GENRE_WEIGHT_WRT_VOTES + DIRECTOR_WEIGHT_WRT_VOTES + CAST_WEIGHT_WRT_VOTES)
    
    return movie_vote

# Gather Data

In [ ]:
# Load the dataset
original_df = pd.read_csv(DATASET_PATH)

# Show first 10 values
original_df.head(10)

In [ ]:
original_df.shape

#### Check for attributes type and null values in dataset

In [ ]:
get_data_frame_quick_insights(original_df)

## 💡Findings:
* There are a few columns with **inconsistent data types** (i.e. some columns have the "object" data type).
* **Stars** column contains hidden info of **Director** and **Cast**.
* There are a few columns with **inconsistent column names**.
* There are a few columns with **inconsistent column values**.
* The **Gross** column has **95%** of its values missing.
* There is a chance for **duplicate entries**.

***Let's work on the above findings first***

# Data Cleaning

In [ ]:
# Make a copy from our original dataframe
data = original_df

In [ ]:
# Check for duplicates
data.duplicated().sum()

In [ ]:
# Dropping the duplication
data.drop_duplicates(inplace=True)
data.shape

In [ ]:
# Rename columns
data.rename(columns={'RunTime': 'RUNTIME', 'Gross': 'GROSS'}, inplace=True)

In [ ]:
# Remove 'Gross' due to its significance absence
remove_columns = ['GROSS']
data = data.drop(columns=remove_columns, axis=1)
data.head(5)

In [ ]:
# Remove rows where YEAR, GENRE, RATING, VOTES, RUNTIME is NaN
df_removed_na_attributes_movies = data[
    (data['YEAR'].isna()) & \
    (data['RATING'].isna()) & \
    (data['VOTES'].isna()) & \
    (data['RUNTIME'].isna())
]
df_removed_na_attributes_movies.shape

In [ ]:
df_removed_na_attributes_movies.head()

*As it can be observed, the entries with missing values in **YEAR**, **RATING**, **RUNTIME**, and **VOTES** account for approximately **5%** of our dataset after removing duplicate entries. It would be better to remove them for now.*

In [ ]:
# Keeping reset of the entries by taking (~) bitwise NOT of NAN entries
data =  data[~
    ((data['YEAR'].isna()) & \
    (data['RATING'].isna()) & \
    (data['VOTES'].isna()) & \
    (data['RUNTIME'].isna()))
]

data.shape

#### Fill Missing values and change column data types

In [ ]:
# [FOR 'GENRE'] Remove '\n' and empty spaces.
# Fill missing values with 'Unknown' as only 0.8% of it is missing
data['GENRE'] = data['GENRE'].str.replace('\n', '').str.strip()
data['GENRE'].fillna('Unknown', inplace=True)

In [ ]:
# Change type and remove '\n' 
data['STARS'] = data['STARS'].astype(str).str.replace('\n', '').str.replace(' ', '').str.strip()

# Expand 'STARS' to add two new attributes into the dataframe
data[['DIRECTOR', 'CAST']] = data['STARS'].apply(extract_director_and_stars_info).tolist()

# Drop 'STARS' column
data = data.drop('STARS', axis=1)
data.head(3)

In [ ]:
# [FOR 'YEAR'] fill missing values
data['YEAR'].fillna('Unknown', inplace=True)

# [FOR 'ONE-LINE'] Change type and remove '\n' 
data['ONE-LINE'] = data['ONE-LINE'].astype(str).str.replace('\n', '')

For now, just fill **VOTES**, **RUNTIME** and **RATING** with 0. Later on we will impute them w.r.t to **DIRECTOR** and **CAST** mean values

In [ ]:
# [FOR 'VOTES'] Remove ',' and NAN
data['VOTES'] = data['VOTES'].str.replace(',', '', regex=True)
data['VOTES'].fillna('0.0', inplace=True)
data['VOTES'] = data['VOTES'].astype(float)

# [FOR 'RUNTIME'] fill missing values
data['RUNTIME'].fillna(0.0, inplace=True)

# [FOR 'RATING'] fill missing values
data['RATING'].fillna(0.0, inplace=True)

Find mean of movies RUNTIME, RATING and VOTES w.r.t each **DIRECTOR**

In [ ]:
# Group by 'Director' and calculate the mean
df_director_mean = data[['DIRECTOR', 'RUNTIME', 'RATING', 'VOTES']].groupby('DIRECTOR').mean().reset_index()
df_director_mean.head(2)

Find mean of movies RUNTIME, RATING and VOTES w.r.t each **CAST**

In [ ]:
# Create a dataframe to store absolute values w.r.t to casts of each movies
movie_casts = []
for index, row in data.iterrows():
    cast_list = row['CAST'].split(',')
    for cast_member in cast_list:
        movie_casts.append({
            'CAST': cast_member, 
            'RUNTIME': row['RUNTIME'], 
            'RATING': row['RATING'], 
            'VOTES': row['VOTES']
        })

df_cast_wise_insigts = pd.DataFrame(movie_casts)

# Group by 'CAST' and calculate the mean
df_cast_mean = df_cast_wise_insigts.groupby('CAST').mean().reset_index()
df_cast_mean.head(2)

Find mean of movies RUNTIME, RATING and VOTES w.r.t each **GENRE**

In [ ]:
# Create a dataframe to store absolute values w.r.t to genre of each movies
movie_genre = []
for index, row in data.iterrows():
    genre_list = row['GENRE'].split(',')
    for genre in genre_list:
        movie_genre.append({
            'GENRE': genre, 
            'RUNTIME': row['RUNTIME'], 
            'RATING': row['RATING'], 
            'VOTES': row['VOTES']
        })
df_genre_wise_insigts = pd.DataFrame(movie_genre)

# Group by 'CAST' and calculate the mean
df_genre_mean = df_genre_wise_insigts.groupby('GENRE').mean().reset_index()
df_genre_mean.head(2)

**Impute RATING values where 0.0:**

*Movie rating highly depends of its genre. Cast and Directors can be equally dependent. Calculating rating with respect to the defined weightage.*

In [ ]:
# Extract row where 'RATING' is equal to 0.0 to impute them
rows_to_update = data[data['RATING'] == 0.0]

for _, row in rows_to_update.iterrows():
    # Initialize values
    director_rating = df_director_mean[df_director_mean['DIRECTOR'] == row['DIRECTOR']].iloc[0]['RATING']
    genre_rating_mean = 0
    cast_rating_mean = 0
    
    # Iterate genre and cast lookup table to extract values
    genre_list = [element.strip() for element in row['GENRE'].split(',')]
    cast_list = [element.strip() for element in row['CAST'].split(',')]
    
    for genre in genre_list:
        genre_rating_mean += df_genre_mean[df_genre_mean['GENRE'] == genre].iloc[0]['RATING']
    for cast in cast_list:
        cast_rating_mean += df_cast_mean[df_cast_mean['CAST'] == cast].iloc[0]['RATING']
    
    # Take mean w.r.t their count
    genre_rating_mean = (genre_rating_mean / len(genre_list)).round(1)
    cast_rating_mean = (cast_rating_mean / len(cast_list)).round(1)
    
    # Update movie 'RATING' attribute value
    rows_to_update['RATING'] = cal_movie_rating(genre_rating_mean, director_rating, cast_rating_mean)
    data.update(rows_to_update)

**Impute VOTES values where 0.0:**

*Movie votes highly depends of its genre and cast. Directors has low dependency. Calculating votes with respect to the defined weightage.*

In [ ]:
# Extract row where 'VOTES' is equal to 0.0 to impute them
rows_to_update = data[data['VOTES'] == 0.0]

for _, row in rows_to_update.iterrows():
    # Initialize values
    director_votes = df_director_mean[df_director_mean['DIRECTOR'] == row['DIRECTOR']].iloc[0]['VOTES']
    genre_votes_mean = 0
    cast_votes_mean = 0
    
    # Iterate genre and cast lookup table to extract values
    genre_list = [element.strip() for element in row['GENRE'].split(',')]
    cast_list = [element.strip() for element in row['CAST'].split(',')]
    
    for genre in genre_list:
        genre_votes_mean += df_genre_mean[df_genre_mean['GENRE'] == genre].iloc[0]['VOTES']
    for cast in cast_list:
        cast_votes_mean += df_cast_mean[df_cast_mean['CAST'] == cast].iloc[0]['VOTES']
    
    # Take mean w.r.t their count
    genre_votes_mean = (genre_votes_mean / len(genre_list)).round(1)
    cast_votes_mean = (cast_votes_mean / len(cast_list)).round(1)
    
    #Update movie 'VOTES' attribute value
    rows_to_update['VOTES'] = cal_movie_votes(genre_votes_mean, director_votes, cast_votes_mean)
    data.update(rows_to_update)

**Impute RUNTIME values where 0.0:**

*Movie runtime highly depends of its genre. Calculating runtime with respect to genre wise mean.*

In [ ]:
# Extract row where 'RUNTIME' is equal to 0.0 to impute them
rows_to_update = data[data['RUNTIME'] == 0.0]

for _, row in rows_to_update.iterrows():
    # Initialize values
    genre_runtime_mean = 0
    
    # Iterate genre and cast lookup table to extract values
    genre_list = [element.strip() for element in row['GENRE'].split(',')]
    
    for genre in genre_list:
        genre_runtime_mean += df_genre_mean[df_genre_mean['GENRE'] == genre].iloc[0]['RUNTIME']
    
    # Take mean w.r.t their count
    genre_runtime_mean = (genre_runtime_mean / len(genre_list)).round(1)
    
    #Update movie 'RUNTIME' attribute value
    rows_to_update['RUNTIME'] = genre_runtime_mean
    data.update(rows_to_update)

In [ ]:
get_data_frame_quick_insights(data)

*Data Cleaning and filling missing values has been completed. Let's begin some data transformation*

# Data Transformation

#### Add conditional column to distinguish between Movie and Series

In [ ]:
# Create a conditional column "Type"
data['TYPE'] = data['YEAR'].apply(lambda x: 'Movie' if pd.notna(x) and pd.Series(x).str.match(REGEX_MOVIE_YEAR).any() else 'Series')
data.head(5)

In [ ]:
df_movies_directors_count = get_directors_count(data[data['TYPE'] == 'Movie']['DIRECTOR'])
df_series_directors_count = get_directors_count(data[data['TYPE'] == 'Series']['DIRECTOR'])

df_movie_cast_count = get_casts_count(data[data['TYPE'] == 'Movie']['CAST'])
df_series_cast_count = get_casts_count(data[data['TYPE'] == 'Series']['CAST'])

df_movie_genre_count = get_genre_count(data[data['TYPE'] == 'Movie']['GENRE'])
df_series_genre_count = get_genre_count(data[data['TYPE'] == 'Series']['GENRE'])

df_genres_count = pd.merge(df_movie_genre_count, df_series_genre_count, on='GENRE', how='outer')
df_genres_count = df_genres_count.fillna(0)
df_genres_count = df_genres_count.rename(columns={'COUNT_x': 'MOVIES_COUNT', 'COUNT_y': 'SERIES_COUNT'})
df_genres_count['SERIES_COUNT'] = df_genres_count['SERIES_COUNT'].astype(int)

### Check insights, correlation and covariance

In [ ]:
df_insights = data[['RATING', 'VOTES', 'RUNTIME']]
df_insights.describe()

In [ ]:
df_insights.corr()

In [ ]:
df_insights.cov()

# Data Visualization

***As our dataset is distributed in 2 categories i.e. Movie and Series. Lets check quick insights between them first.***

In [ ]:
movie_type_counts = data['TYPE'].value_counts()

# Create a pie chart
plt.pie(
    movie_type_counts, 
    labels=movie_type_counts.index, 
    autopct='%1.1f%%', 
    colors=['#A6B1E1','#F4EEFF'],
    wedgeprops={'edgecolor': '#424874', 'linewidth': 1.2},
    startangle=140
) 

plt.title('Movie VS Series')
plt.show()

In [ ]:
# Data parameters
genre_types = df_genres_count['GENRE']
dataset1 = df_genres_count['MOVIES_COUNT']
dataset2 = df_genres_count['SERIES_COUNT']
bar_width = 0.35
x_pos = range(len(genre_types))

# Create a figure and axis
fig, ax = plt.subplots(figsize=(12, 6))

# Create bar plots for movies and series
ax.bar(x_pos, dataset1, width=bar_width, label='Movies', align='center', color='#A6B1E1', edgecolor='#424874', linewidth=1.6)
ax.bar([x + bar_width for x in x_pos], dataset2, width=bar_width, label='Series', align='center', color='#424874', edgecolor='#424874', linewidth=1.6)

# Customize the x-axis labels
ax.set_xticks([x + bar_width / 2 for x in x_pos])
ax.set_xticklabels(genre_types, rotation=90)

# Add labels and title
ax.set_xlabel('Genres')
ax.set_ylabel('Count')
ax.set_title('Movies VS Series Genre Count')
ax.legend()

# Display the plot
plt.tight_layout()
plt.show()


***Lets looks for the distribution of 'Rating' using a histogram***

In [ ]:
# Filter data for movies and series separately
movie_data = data[data['TYPE'] == 'Movie']
series_data = data[data['TYPE'] == 'Series']

# Set plot size and style
plt.figure(figsize=(18, 5))
plt.style.use('seaborn-whitegrid')

# Plotting Movie Ratings Histogram
plt.subplot(131)
plt.hist(movie_data['RATING'], bins=30, color='#A6B1E1', edgecolor='#424874', linewidth=1.5)
plt.title('Movie Ratings')

# Calculate and display median, mode, and mean for movie ratings
median_movie = np.median(movie_data['RATING'])
mode_movie = movie_data['RATING'].mode().iloc[0]
mean_movie = np.mean(movie_data['RATING'])
plt.axvline(median_movie, color='red', linestyle='dashed', linewidth=2, label=f'Median: {median_movie:.2f}')
plt.axvline(mode_movie, color='green', linestyle='dashed', linewidth=2, label=f'Mode: {mode_movie:.2f}')
plt.axvline(mean_movie, color='purple', linestyle='dashed', linewidth=2, label=f'Mean: {mean_movie:.2f}')
plt.legend()

# Plotting Series Ratings Histogram
plt.subplot(132)
plt.hist(series_data['RATING'], bins=30, color='#A6B1E1', edgecolor='#424874', linewidth=1.5)
plt.title('Series Ratings')

# Calculate and display median, mode, and mean for series ratings
median_series = np.median(series_data['RATING'])
mode_series = series_data['RATING'].mode().iloc[0]
mean_series = np.mean(series_data['RATING'])
plt.axvline(median_series, color='red', linestyle='dashed', linewidth=2, label=f'Median: {median_series:.2f}')
plt.axvline(mode_series, color='green', linestyle='dashed', linewidth=2, label=f'Mode: {mode_series:.2f}')
plt.axvline(mean_series, color='purple', linestyle='dashed', linewidth=2, label=f'Mean: {mean_series:.2f}')

# Display legends
plt.legend()

# Adjust layout for better spacing
plt.tight_layout()

# Display the plot
plt.show()


In [ ]:
plt.figure(figsize=(6,6))
custom_cmap = mcolors.LinearSegmentedColormap.from_list("custom", ['#F4EEFF', '#A6B1E1', '#424874'])
sns.heatmap(df_insights.corr(), annot=True, cbar=True, annot_kws={'size': 14}, cmap=custom_cmap)
plt.show